<a href="https://colab.research.google.com/github/sunyingjian/-Logging-related-network/blob/master/%E9%87%87%E7%94%A8%E5%A4%9A%E5%85%83%E7%BA%BF%E6%80%A7%2C%E5%B2%AD%E5%9B%9E%E5%BD%92%2CSVR%2CGBDT%E5%AE%9E%E7%8E%B0%E5%AD%94%E9%9A%99%E5%BA%A6%E5%9B%9E%E5%BD%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### <font size=5px color="red">✦ *Google Colab 突破90分钟自动断开:</font>
<p><font size=3px > 每60分钟自动运行代码以刷新90分钟断开限制. 打开 developer-settings (在你的浏览器) 快速健 Ctrl+Shift+I 然后按console 输入以下代码 Enter. ( mac 按 Option+Command+I)</p><b>复制以下隐藏代码粉贴在浏览器console！！不要关闭浏览器以免失效</b>

<code>function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-connect-button").click() 
}setInterval(ClickConnect,6000)</code>

In [1]:
#@markdown <h3>← 输入了代码后运行以防止断开</h>


import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


#引入数据

In [2]:
! git clone https://github.com/sunyingjian/numpy-.git

Cloning into 'numpy-'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 106 (delta 6), reused 0 (delta 0), pack-reused 89
Receiving objects: 100% (106/106), 3.90 MiB | 21.58 MiB/s, done.
Resolving deltas: 100% (39/39), done.


#数据处理

In [3]:
%matplotlib inline
#%matplotlib inline 可以在Ipython编译器里直接使用，功能是可以内嵌绘图，并且可以省略掉plt.show()这一步。
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pandas import set_option
set_option("display.max_rows", 10)#设置要显示的默认行数，显示的最大行数是10
pd.options.mode.chained_assignment = None #为了在增加列表行数的时候防止出现setting with copy warning
data = pd.read_csv("/content/numpy-/预测核磁共振和孔隙度测井数据集/WR313H_logs.csv")
data=data.dropna(how='any')
data.shape

(6540, 5)

In [4]:
porosity_labels = data['Porosity'].values
porosity_data = data.drop([ 'Depth','Porosity'], axis=1)
porosity_data.describe()

,GR,Caliper,Ring_resistivity
count,6540.000000,6540.000000,6540.000000
mean,100.697470,21.794128,2.631533
std,10.971742,0.600965,15.522354
min,35.249556,21.045932,0.330400
25%,94.774698,21.484780,1.582150
50%,102.349457,21.698458,1.848500
75%,108.030412,21.941599,2.095000
max,126.619332,30.664150,649.611600


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
        porosity_data, porosity_labels, test_size=0.2, random_state=42)
X_train.shape

(5232, 3)

#先看下相关性

In [6]:
from pandas import set_option
import seaborn as sns
from sklearn import  model_selection
import statsmodels.api as sm
import statsmodels.formula.api as smf

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
model = smf.ols(formula='Porosity~GR+Caliper+Ring_resistivity',data=data).fit()
print(f'模型的偏回归系数为{model.params}\n')

模型的偏回归系数为Intercept          -0.201605
GR                  0.000212
Caliper             0.025088
Ring_resistivity   -0.000068
dtype: float64



In [8]:
#计算建模中的因变量均值
ybar = data.Porosity.mean()
#统计变量个数与观测个数
p=model.df_model
n=data.shape[0]
##计算回归离差平方和
RSS=np.sum((model.fittedvalues-ybar)**2)
RSS

1.531428215533501

In [9]:
#计算误差平方和
ESS = np.sum(model.resid**2)
ESS

38.46811521714825

In [10]:
#计算F值
F = (RSS/p)/(ESS/(n-p-1))
F

86.73343070596627

In [11]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Porosity   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     86.73
Date:                Sun, 02 Aug 2020   Prob (F-statistic):           4.97e-55
Time:                        14:13:10   Log-Likelihood:                 7514.4
No. Observations:                6540   AIC:                        -1.502e+04
Df Residuals:                    6536   BIC:                        -1.499e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -0.2016      0.036     -5.640      0.000      -0.272      -0.132
GR                   0.0002   8.84e-05      2.396      0.017    3.85e-05       0.000
Caliper              0.0251      0.002     15.887      0.000       0.022       0.028
Ring_resistivity -6.774e-05   6.25e-05     -1.084      0.279      -0.000    5.48e-05
==============================================================================
Omnibus:                     3780.366   Durbin-Watson:                   0.018
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            40865.082
Skew:                           2.601   Prob(JB):                         0.00
Kurtosis:                      14.086   Cond. No.                     3.91e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.91e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#线性回归

In [28]:
import sklearn
from sklearn import metrics
from sklearn import linear_model
import statsmodels.api as sm
from sklearn.metrics import r2_score

reg = linear_model.LinearRegression()
reg.fit(X_train,y_train)
y_pred = reg.predict(X_test)
print(f'MSE={metrics.mean_absolute_error(y_test,y_pred)}')
print(f'可决系数R2:{r2_score(y_test,y_pred)}')

MSE=0.052455566216975204
可决系数R2:0.04585913996340052


##岭回归

In [29]:
reg1 = linear_model.Ridge(alpha=0.5)
reg1.fit(X_train,y_train)
y_pred = reg1.predict(X_test)
print('Mean squared error: %f'
      % metrics.mean_squared_error(y_test, y_pred))
print(f'可决系数R2:{r2_score(y_test,y_pred)}')

Mean squared error: 0.006704
可决系数R2:0.04585351695210571


#SVR支持向量回归

In [30]:
from sklearn import svm
clf2 = svm.SVR()
clf2.fit(X_train,y_train)
y_pred = clf2.predict(X_test)
print('Mean squared error: %f'
      % metrics.mean_squared_error(y_test, y_pred))
print(f'可决系数R2:{r2_score(y_test,y_pred)}')

Mean squared error: 0.006114
可决系数R2:0.1297301490962368


#GBDT回归

In [31]:
from sklearn.ensemble import GradientBoostingRegressor
est = GradientBoostingRegressor(n_estimators=100,learning_rate=0.1,max_depth=1,random_state=0,
                                loss='ls').fit(X_train,y_train)
y_pred = est.predict(X_test)
print('Mean squared error: %f'
      % metrics.mean_squared_error(y_test, y_pred))
print(f'可决系数R2:{r2_score(y_test,y_pred)}')

Mean squared error: 0.002324
可决系数R2:0.6692259002343055


#画图